# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa2f4f8cca0>
├── 'a' --> tensor([[ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619]])
└── 'x' --> <FastTreeValue 0x7fa2f4f8cb50>
    └── 'c' --> tensor([[ 0.0937,  1.7529, -1.5770, -0.6113],
                        [-0.4601,  1.1002,  1.3489, -0.2936],
                        [ 0.1501,  1.9876, -0.4685, -1.2364]])

In [4]:
t.a

tensor([[ 1.2057, -0.4213, -0.9816],
        [-0.5064, -1.0093,  0.0619]])

In [5]:
%timeit t.a

63.3 ns ± 0.0943 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa2f4f8cca0>
├── 'a' --> tensor([[ 0.4395,  2.0048,  1.5103],
│                   [-1.2120, -0.6876,  2.1990]])
└── 'x' --> <FastTreeValue 0x7fa2f4f8cb50>
    └── 'c' --> tensor([[ 0.0937,  1.7529, -1.5770, -0.6113],
                        [-0.4601,  1.1002,  1.3489, -0.2936],
                        [ 0.1501,  1.9876, -0.4685, -1.2364]])

In [7]:
%timeit t.a = new_value

69.5 ns ± 0.0289 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619]]),
    x: Batch(
           c: tensor([[ 0.0937,  1.7529, -1.5770, -0.6113],
                      [-0.4601,  1.1002,  1.3489, -0.2936],
                      [ 0.1501,  1.9876, -0.4685, -1.2364]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2057, -0.4213, -0.9816],
        [-0.5064, -1.0093,  0.0619]])

In [11]:
%timeit b.a

57.8 ns ± 0.0444 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0136, -0.8212,  0.4858],
               [ 0.6165,  0.4698, -0.6751]]),
    x: Batch(
           c: tensor([[ 0.0937,  1.7529, -1.5770, -0.6113],
                      [-0.4601,  1.1002,  1.3489, -0.2936],
                      [ 0.1501,  1.9876, -0.4685, -1.2364]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.669 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 44.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 413 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa2f4f8ce50>
├── 'a' --> tensor([[[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]],
│           
│                   [[ 1.2057, -0.4213, -0.9816],
│                    [-0.5064, -1.0093,  0.0619]]])
└── 'x' --> <FastTreeValue 0x7fa2f4f8cd90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 84.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa244252f40>
├── 'a' --> tensor([[ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619],
│                   [ 1.2057, -0.4213, -0.9816],
│                   [-0.5064, -1.0093,  0.0619]])
└── 'x' --> <FastTreeValue 0x7fa2f4fb6250>
    └── 'c' --> tensor([[ 0.0937,  1.7529, -1.5770, -0.6113],
                        [-0.4601,  1.1002,  1.3489, -0.2936],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 49.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]],
       
               [[ 1.2057, -0.4213, -0.9816],
                [-0.5064, -1.0093,  0.0619]]]),
    x: Batch(
           c: tensor([[[ 0.0937,  1.7529, -1.5770, -0.6113],
                       [-0.4601,  1.1002,  1.3489, -0.2936],
                       [ 0.1501,  1.9876, -0.4685, -1.2364]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619],
               [ 1.2057, -0.4213, -0.9816],
               [-0.5064, -1.0093,  0.0619]]),
    x: Batch(
           c: tensor([[ 0.0937,  1.7529, -1.5770, -0.6113],
                      [-0.4601,  1.1002,  1.3489, -0.2936],
                      [ 0.1501,  1.9876, -0.4685, -1.2364],
                      [ 0.0937,  1.7529, -1.5770, -0.6113],
                      [-0.4601,  

In [28]:
%timeit Batch.cat(batches)

146 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
